# GPU Acceleration with PyTorch: A Tensor Operations Primer

This notebook explores PyTorch's power as an efficient library for tensor operations, which are fundamental to computational modeling in AI and beyond. Rather than focusing on neural networks, we will demonstrate how PyTorch enables high-performance scientific computing through three compelling examples.

Tensors are multi-dimensional arrays that generalize scalars (0D), vectors (1D), and matrices (2D). They are ideal for numerical computing because they provide a structured, contiguous memory layout that enables efficient access and operations. Unlike Python lists, tensors support vectorized operations—applying functions to entire arrays at once without explicit loops—which reduces overhead and leverages hardware optimizations. Additionally, features like broadcasting allow tensors of different shapes to be combined implicitly, making code concise and less error-prone.

Many real-world problems involve massive datasets or computations. CPUs, with their handful of powerful cores, excel at sequential tasks but bottleneck on parallel workloads. GPUs, originally designed for graphics rendering, have thousands of smaller cores that can execute the same operation on many data elements simultaneously. This massive parallelism is perfect for tensor operations. Without GPUs, large-scale modeling would be impractically slow; with them, we can achieve 10-100x speedups. This phenomenon, where GPU advancements for gaming accidentally empowered parallel computing, is often called the "hardware lottery" that propelled modern AI forward.

We'll explore these concepts through non-NN use cases: Monte Carlo methods, cellular automata, and fractal generation. We will use only base PyTorch operations and functional modules, steering clear of any `nn` modules to keep the focus on raw tensor capabilities.

A key note on GPU performance: The first execution of an operation on the GPU can be slower because PyTorch performs just-in-time (JIT) compilation of CUDA kernels and allocates memory. This setup overhead happens only once. To demonstrate true performance, we will include a "warmup" pass—running the computation once without timing it—before measuring.

## Section 1: PyTorch Basics – Tensors and Devices

Let's start with a quick setup. We will import the essentials and identify the available compute devices.

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import time  # For timing comparisons

# For animation
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
plt.rcParams['animation.html'] = 'jshtml' # Configure for notebook display

In [ ]:
# Setup our compute devices
cpu_device = torch.device("cpu")
gpu_device = None
if torch.cuda.is_available():
    gpu_device = torch.device("cuda")
    print(f"GPU is available: {torch.cuda.get_device_name(0)}")
else:
    print("GPU not available, all operations will run on CPU.")

PyTorch lets you move tensors to a GPU with `.to(device)`, and operations on that tensor will automatically parallelize. For example, let's time matrix multiplication, a core operation in nearly all scientific computing:

In [ ]:
def matmul_example(dev):
    A = torch.randn(2000, 2000, device=dev)
    B = torch.randn(2000, 2000, device=dev)
    C = torch.matmul(A, B)  # Trigger computation
    if dev.type == 'cuda':
        torch.cuda.synchronize()  # Ensure GPU finishes
    return C

# Warmup if GPU is available
if gpu_device:
    _ = matmul_example(dev=gpu_device)

# Time on CPU
start_cpu = time.time()
_ = matmul_example(dev=cpu_device)
print(f"CPU Time: {time.time() - start_cpu:.4f}s")

# Time on GPU (post-warmup), if available
if gpu_device:
    start_gpu = time.time()
    _ = matmul_example(dev=gpu_device)
    print(f"GPU Time: {time.time() - start_gpu:.4f}s")

## Section 2: Monte Carlo Methods – Parallel Sampling

Monte Carlo methods rely on repeated random sampling to obtain numerical results, especially when deterministic solutions are infeasible. A classic example is approximating π by randomly scattering points in a unit square and finding the proportion that falls within an inscribed quarter-circle. This proportion approaches π/4 as the number of points increases.

This problem is perfectly suited for parallelization, as each random sample is independent. In PyTorch, we can generate all samples simultaneously in a large tensor and perform the calculations without any loops.

In [ ]:
def approximate_pi(num_samples=10000000, dev=cpu_device):
    # Generate random points: two tensors for x and y coords
    x = torch.rand(num_samples, device=dev)
    y = torch.rand(num_samples, device=dev)
    
    # Element-wise operations: compute distance squared
    inside = (x**2 + y**2) <= 1.0  # Boolean tensor
    
    # Reduction: count trues in parallel
    num_inside = torch.sum(inside.float())
    
    pi_approx = (num_inside / num_samples) * 4
    if dev.type == 'cuda':
        torch.cuda.synchronize()
    return pi_approx.item()

# Warmup on GPU
if gpu_device:
    _ = approximate_pi(dev=gpu_device)

# Time on CPU
start_cpu = time.time()
pi_cpu = approximate_pi(dev=cpu_device)
print(f"CPU π ≈ {pi_cpu:.6f}, Time: {time.time() - start_cpu:.4f}s")

# Time on GPU (post-warmup)
if gpu_device:
    start_gpu = time.time()
    pi_gpu = approximate_pi(dev=gpu_device)
    print(f"GPU π ≈ {pi_gpu:.6f}, Time: {time.time() - start_gpu:.4f}s")

As the code demonstrates, more samples yield a more accurate result at the cost of computation time. You can explore this trade-off by re-running the cell above with different values for `num_samples`. For instance, try a smaller value like `100000` and a much larger one like `100000000` to see the effect on both the π estimate and the execution time.

### Visualization: The Random Samples

In [ ]:
num_viz = 10000
x_viz = torch.rand(num_viz)
y_viz = torch.rand(num_viz)
inside_viz = (x_viz**2 + y_viz**2) <= 1.0

plt.figure(figsize=(6, 6))
plt.scatter(x_viz[inside_viz], y_viz[inside_viz], color='blue', s=1, label='Inside')
plt.scatter(x_viz[~inside_viz], y_viz[~inside_viz], color='red', s=1, label='Outside')
plt.gca().set_aspect('equal')
plt.title(f"Monte Carlo π Approximation (Samples: {num_viz})")
plt.legend()
plt.show()

### Visualization: Convergence of π

The accuracy of the approximation improves with more samples. We can visualize this convergence, which illustrates the Law of Large Numbers.

In [ ]:
sample_sizes = torch.logspace(3, 7, 20).int()
pi_estimates = []

for n in sample_sizes:
    # Use CPU for this visualization as individual runs are fast
    pi_val = approximate_pi(num_samples=n.item(), dev=cpu_device)
    pi_estimates.append(pi_val)

plt.figure(figsize=(10, 6))
plt.plot(sample_sizes, pi_estimates, 'o-', label='π Estimate')
plt.axhline(y=np.pi, color='r', linestyle='--', label='Actual π')
plt.xscale('log')
plt.xlabel('Number of Samples (log scale)')
plt.ylabel('Estimated Value of π')
plt.title('Convergence of Monte Carlo π Approximation')
plt.legend()
plt.grid(True, which="both", ls="--")
plt.show()

## Section 3: Cellular Automata – Grid-Based Simulations

Cellular automata are models consisting of a grid of cells, each evolving over time according to fixed rules based on its neighbors. They demonstrate how complex, emergent behaviors can arise from simple local interactions. Conway's Game of Life is a famous example.

The rules for the Game of Life are:
1.  **Underpopulation:** A live cell with fewer than 2 live neighbors dies.
2.  **Survival:** A live cell with 2 or 3 live neighbors stays alive.
3.  **Overpopulation:** A live cell with more than 3 live neighbors dies.
4.  **Reproduction:** A dead cell with exactly 3 live neighbors becomes alive.

This process can be efficiently implemented using a 2D convolution to count the neighbors of every cell simultaneously. The update rules are then applied as a set of parallel, element-wise tensor operations.

In [ ]:
import torch.nn.functional as F

def game_of_life(steps=100, grid_size=100, dev=cpu_device, initial_prob=0.5):
    grid = (torch.rand(grid_size, grid_size, device=dev) > initial_prob).float()
    kernel = torch.ones(3, 3, device=dev); kernel[1,1] = 0
    grids = [grid.clone().cpu().numpy()]
    for _ in range(steps):
        neighbors = F.conv2d(grid.unsqueeze(0).unsqueeze(0), kernel.unsqueeze(0).unsqueeze(0), padding=1).squeeze()
        rule1 = (neighbors == 3) # Reproduction
        rule2 = ((neighbors == 2) & (grid == 1)) # Survival
        grid = (rule1 | rule2).float()
        grids.append(grid.clone().cpu().numpy())
    if dev.type == 'cuda': torch.cuda.synchronize()
    return grids

if gpu_device: _ = game_of_life(dev=gpu_device)
start_cpu = time.time()
grids_cpu = game_of_life(dev=cpu_device)
print(f"CPU Time: {time.time() - start_cpu:.4f}s")
if gpu_device:
    start_gpu = time.time()
    grids_gpu = game_of_life(dev=gpu_device)
    print(f"GPU Time: {time.time() - start_gpu:.4f}s")

### Visualization: The Evolution of Life

In [ ]:
fig = plt.figure(figsize=(6, 6))
im = plt.imshow(grids_cpu[0], cmap='binary')
plt.axis('off')
plt.title("Game of Life Evolution")

def animate(i):
    im.set_data(grids_cpu[i])
    plt.gca().set_title(f'Step {i}')
    return [im]

anim = FuncAnimation(fig, animate, frames=len(grids_cpu), interval=50, blit=True)
plt.close()
HTML(anim.to_jshtml())

The `initial_prob` parameter in the `game_of_life` function sets the initial density of live cells, which can dramatically alter the system's evolution. To observe this, you can modify the function call to experiment with different densities. A sparse world (e.g., `initial_prob=0.8`, for a 20% density) often dies out quickly, while a very dense world (e.g., `initial_prob=0.2`, for an 80% density) might collapse into stable patterns or chaos. Try these values and see how the animation changes.

## Section 4: Fractal Generation – Iterative Computations

Fractals are geometric shapes that exhibit self-similarity at every scale. The Mandelbrot set is a famous fractal in the complex plane, defined by iterating the function `z = z² + c` for every point `c` on the plane. If the value of `z` remains bounded, the point `c` is in the set.

Generating this set is computationally intensive, as the iteration must be performed for every pixel in the final image. However, since the calculation for each point is independent, the entire grid of points can be processed in parallel on a GPU.

In [ ]:
def mandelbrot(width=800, height=600, x_center=-0.75, y_center=0, zoom=1, max_iter=100, dev=cpu_device):
    x_width = 3.5 / zoom
    y_height = 2.0 / zoom * height / width
    x = torch.linspace(x_center - x_width / 2, x_center + x_width / 2, width, device=dev)
    y = torch.linspace(y_center - y_height / 2, y_center + y_height / 2, height, device=dev)
    real, imag = torch.meshgrid(x, y, indexing='xy')
    c = real + imag * 1j
    z = torch.zeros_like(c)
    escaped = torch.zeros_like(real, dtype=torch.int)
    for i in range(max_iter):
        z = z**2 + c
        not_escaped = torch.abs(z) <= 2
        escaped[not_escaped] = i + 1
    if dev.type == 'cuda': torch.cuda.synchronize()
    return escaped.cpu().numpy()

if gpu_device: _ = mandelbrot(dev=gpu_device)
start_cpu = time.time()
fractal_cpu = mandelbrot(dev=cpu_device)
print(f"CPU Time: {time.time() - start_cpu:.4f}s")
if gpu_device:
    start_gpu = time.time()
    fractal_gpu = mandelbrot(dev=gpu_device)
    print(f"GPU Time: {time.time() - start_gpu:.4f}s")

### Visualization: The Mandelbrot Set

In [ ]:
plt.figure(figsize=(10, 7))
plt.imshow(fractal_cpu, cmap='hot', extent=[-2.5, 1.0, -1.5, 1.5])
plt.title("Mandelbrot Set")
plt.colorbar(label='Iteration Count for Escape')
plt.show()

The level of detail in the fractal's boundary is controlled by the `max_iter` parameter. More iterations resolve finer patterns but increase the computational cost. Feel free to experiment by changing this value in the `mandelbrot` function call. A lower value like `50` will render faster but with less detail, while a higher value like `250` will reveal more intricate structures at the cost of longer execution time.

### Visualization: Zooming into the Fractal

The most fascinating property of fractals is their infinite complexity. We can zoom into a region on the boundary of the set to reveal intricate, self-repeating patterns. This requires high resolution and many iterations, a task well-suited for a GPU.

In [ ]:
# Coordinates for a visually interesting region ('Seahorse Valley')
x_center, y_center = -0.745, 0.186
zoom = 200
max_iter_zoom = 500

# Use the faster device for this intensive computation
compute_device = gpu_device if gpu_device else cpu_device

fractal_zoom = mandelbrot(
    width=1200, height=800, 
    x_center=x_center, y_center=y_center, 
    zoom=zoom, max_iter=max_iter_zoom, 
    dev=compute_device
)

plt.figure(figsize=(12, 8))
plt.imshow(fractal_zoom, cmap='magma')
plt.title(f'Mandelbrot Set (Zoomed to {zoom}x)')
plt.axis('off')
plt.show()

## Conclusion: The Foundation for AI

Through these examples, we've seen PyTorch tensors excel at Monte Carlo sampling, grid-based simulations, and iterative fractal generation. All three leverage parallel operations like element-wise math, reductions, and convolutions, which are accelerated dramatically by GPUs.

This is the fundamental principle that enables modern AI. Neural networks are composed of large tensors of weights, and their core operations—like matrix multiplications during a forward pass and gradient calculations during backpropagation—are precisely the kinds of parallel tasks that GPUs are designed to handle. Without GPU-accelerated tensor libraries like PyTorch, training today's large-scale models would be computationally infeasible.